# Run Spectractor from notebooks

- adapted from https://github.com/stubbslab/PCWG-AuxTel/blob/main/Run_Spectractor_example.ipynb

- author : Sylvie Dagoret-campagne
- affiliation : IJCLab/IN2P3/CNRS
- update : September 2021 13th
- DM-Stack version : **w_2021_36**
- Big CPU

In [19]:
# wide scan : HD 160617 
all_myseq_holo=range(234,310)
# narrow scan :  HD 160617 
#all_myseq_holo=range(317,365)
all_myseq=all_myseq_holo

ExpoNum=308

In [20]:
#Check your stack container version matches the setup instructions!
version = ! eups list -s lsst_distrib
print(f'You are running {version[0].split()[1]} of lsst_distrib')

You are running current of lsst_distrib


Check your packages are setup, as per the setup instructions. Each of the ones setup should appear here. If they don't, you've missed the line in your .user_setups file

In [21]:
! eups list -s | grep LOCAL

atmospec              LOCAL:/home/dagoret/repos/atmospec 	setup
daf_butler            LOCAL:/home/dagoret/repos/daf_butler 	setup
eups                  LOCAL:/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-0.7.0/eups 	setup
rapid_analysis        LOCAL:/home/dagoret/repos/rapid_analysis 	setup


Redirect the logger outputs (stack and Spectractor) to the notebook:

In [22]:
from lsst.log.utils import enable_notebook_logging
enable_notebook_logging()

Make a bulter, pick an image, and make a data reference to it:

In [23]:
import lsst.daf.persistence as dafPersist

#After mfl created /project/shared/auxTel/rerun/dagoret (2021/09/09):
#repoDir='/project/shared/auxTel/rerun/dagoret/output'
#repoDir='/project/shared/auxTel/rerun/dagoret'
repoDir='/project/shared/auxTel/rerun/dagoret/outputspectr_scan2021_July'
butler=  dafPersist.Butler(repoDir)

dataId = {'dayObs': '2021-07-07', 'seqNum': ExpoNum}
# good
#dataId = {'dayObs': '2021-07-07', 'seqNum': 330}
dataRef = butler.dataRef('raw', **dataId)

LsstCamMapper WARN: Unable to find valid calib root directory
CameraMapper INFO: Loading exposure registry from /lsstdata/offline/instrument/LATISS/gen2repo/registry.sqlite3
CameraMapper INFO: Loading calib registry from /project/shared/auxTel/CALIB/calibRegistry.sqlite3
CameraMapper INFO: Loading calib registry from /project/shared/auxTel/CALIB/calibRegistry.sqlite3


In [24]:
type(dataId)

dict

Instantiate the task, set our config options

In [25]:
from lsst.atmospec import ProcessStarTask

config = ProcessStarTask.ConfigClass()
config.doDisplayPlots = True  # show the plots in the notebook
config.spectractorDebugMode = True  # make all the debug plots along the way
config.binning = 4

# pretty minimal ISR because some things we don't have the calib products available for
# most of this would be picked up automatically if running from the command line from config files
config.isr.doLinearize = False
config.isr.doDark = False
config.isr.doFlat = False
config.isr.doFringe = False
config.isr.doDefect = True
config.isr.doCrosstalk = False
config.isr.doSaturationInterpolation = False

task = ProcessStarTask(config=config)

Set this so that the plots pop up in the notebook

In [26]:
%matplotlib inline

Run, and watch the debug plots roll in. If you've selected a writable rerun above, the result will also be butler.put() in there so you can butler.get() it later, but we can also catch the result as it's returned by the runDataRef() method

In [27]:
result = task.runDataRef(dataRef)

2021-09-14 16:37:05,309 obs.lsst.assembly logCmd               WARNING (minimum=(0, 0), maximum=(543, 2047)): (minimum=(0, 0), maximum=(575, 2047))
2021-09-14 16:37:06,630 astro_metadata_translator.headers fix_header           CRITICAL Ignoring translator header fixup of LATISS None: 'MJD-OBS not found'
2021-09-14 16:37:10,784 processStar.isr roughZeroPoint       WARNING No rough magnitude zero point defined for filter RG610~holo4_003.
2021-09-14 16:37:13,657 LsstLatissMapper _computeCcdExposureId WARNING Controller unknown, using 'C'
2021-09-14 16:38:04,797 processStar runAstrometry        WARNING Failed to find an acceptable match


IndexError: list index out of range

In [ ]:
print("hello")

In [ ]:
print(result)

In [ ]:
ExpoNum